In [ ]:
!pip install unstructured[pdf]

In [35]:
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import CharacterTextSplitter
import unstructured
from PyPDF2 import PdfReader 

In [37]:
url = 'https://mohua.gov.in/upload/uploadfiles/files/PMSVANidhi%20Guideline_English.pdf'

In [33]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
loader = UnstructuredURLLoader([url])
loader.load()
#texts = text_splitter.split_text(loader.load)
#embeddings = OpenAIEmbeddings()
#embedding_vectors = embeddings.embed_documents(texts)
#faiss_index = FAISS.from_documents(embedding_vectors)
#faiss_index


Error fetching or processing https://mohua.gov.in/upload/uploadfiles/files/PMSVANidhi%20Guideline_English.pdf, exception: partition_pdf is not available. Install the pdf dependencies with pip install "unstructured[pdf]"


[]

In [ ]:
def load_pdf(pdf):
    # Create a PdfReader object to read the PDF file
    pdf_reader = PdfReader(pdf)
    
    # Initialize an empty string to store the extracted text from the PDF
    text = ""
    
    # Iterate through each page in the PDF
    for page in pdf_reader.pages:
        # Extract the text content from the current page and append it to the 'text' variable
        text += page.extract_text()
    
    # Use the text_to_doc_splitter function to split the extracted text into a document
    document = text_to_doc_splitter(text)
    
    # Return the resulting document
    return document   

load_pdf(url)

In [7]:
import openai, requests, pickle, os, streamlit as st
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA 

# Load OpenAI API key from .config file
def load_api_key():
    try:
        with open(r'C:\Users\akash\Documents\Haqdarshak\Work\Scheme_Research_Application\.config', 'r') as file:
            api_key = file.readline().strip()
            os.environ["OPENAI_API_KEY"] = api_key # Creating virtual environment to store Open AI API key virtuallly.
            return api_key
    except Exception as e:
        st.error(f"Error loading API key: {e}")
        return None

api_key = load_api_key()
openai.api_key = api_key

def fetch_article_content(url):
    try:
        response = requests.get(url)
        response.raise_for_status() 
        return response.text
    except Exception as e:
        st.error(f"Error fetching content from {url}: {e}")
        return ""

# Defining a function to vectorize the text for embedding
def generate_embeddings(text, api_key):
    try:
        text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=10, separator="\n,.:")
        texts = text_splitter.split_text(text)
        embeddings = OpenAIEmbeddings(openai_api_key=api_key)
        return embeddings.embed_documents(texts)
    except Exception as e:
        st.error(f"Error generating embeddings: {e}")
        return e

# Defining a function to store indexes of embeddings
def save_faiss_index(index, path):
    try:
        with open(path, 'wb') as f:
            pickle.dump(index, f)
    except Exception as e:
        st.error(f"Error saving FAISS index: {e}")

# Defining a function to load indexes from the pickle store
def load_faiss_index(path):
    try:
        with open(path, 'rb') as f:
            return pickle.load(f)
    except Exception as e:
        st.error(f"Error loading FAISS index: {e}")
        return None

# Defining a function to generate summary of the document
def get_summary(faiss_index, openai_api_key):
   llm = ChatOpenAI(temperature=0.5, model_name='gpt-4o', openai_api_key = openai_api_key)
   pdf_qa = RetrievalQA.from_chain_type(
       llm,
       retriever=faiss_index.as_retriever(search_kwargs={'k': 4}),
       chain_type="stuff",
       )
   # Defining the Query. 
   query = """ Write a summary for the document passed to you. You are getting a PDF document and your job \
        is to interpret the information in the document and to provide a summary. This summary should highlight \
        key points like Scheme Benefits, Scheme Application Process, Eligibility, and Documents required"""
    
    # Executing the RetreivalIQA model with the defined query. 
   result = pdf_qa.run(query)

   return result



In [8]:
generate_embeddings(fetch_article_content("https://github.com/JPSchloss/Open-AI-Summarizer-App/blob/main/ChatGPTSummarizer.pdf"), api_key)
#generate_embeddings(fetch_article_content("https://arxiv.org/pdf/1706.03762"), api_key)


openai.RateLimitError("Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}")

In [ ]:
# Title of web interface.
st.title("Scheme Research Application")

# Getting URL of the file from user.
urls = st.sidebar.text_area("Enter URLs (one per line):")

# Creating action button "Process URL" and steps to follow.
if st.sidebar.button("Process URLs"):
    urls = urls.split("\n")
    all_texts = []
    for url in urls:
        with st.spinner(f"Fetching content from: {url}"):
            content = fetch_article_content(url)
        if content:
            all_texts.append(content)
    
    combined_text = "\n".join(all_texts)


    if combined_text:
        with st.spinner("Generating embeddings..."):
            embedding_vectors = generate_embeddings(combined_text, api_key)
        
        if embedding_vectors:
            st.write("Indexing with FAISS...")
            faiss_index = FAISS.from_documents(embedding_vectors)
            save_faiss_index(faiss_index, r'C:\Users\akash\Documents\Haqdarshak\Work\Scheme_Research_Application\faiss_store_openai.pkl')
            st.success("Processing complete and FAISS index saved.")
            with st.spinner("Generating summary..."):
                summary = get_summary(faiss_index, api_key)
                st.write(summary)

        else:
            st.error("Failed to generate embeddings.")
    else:
        st.error("Failed to fetch or combine content from URLs.")

question = st.text_input("Enter your question:")
if st.button("Ask Question"):
    faiss_index = load_faiss_index(r'C:\Users\akash\Documents\Haqdarshak\Work\Scheme_Research_Application\faiss_store_openai.pkl')
    if faiss_index and question:
        st.write("Fetching answer...")
        similar_docs = faiss_index.similarity_search(question)
        if similar_docs:
            answer = similar_docs[0]['text']
            st.write(f"Answer: {answer}")
            st.write(f"Source URL: {similar_docs[0]['metadata']['source']}")
        else:
            st.error("No relevant documents found.")
    else:
        st.error("Failed to load FAISS index or no question provided.")